## Auto mode (single uav)

This is for testing propouses

In [ ]:
# Custom modules
from config import Color
from helpers import clean  #, local2global
from mavlink.customtypes.location import ENUPose, GRAPose
from plan import Plan
from simulator import (
    QGC,
    ConfigGazebo,
    ConfigQGC,
    ConfigVis,
    Gazebo,
    NoneVisualizer,
    Simulator,
)

clean()

## Simulation Configuration

In [ ]:
gra_origin = GRAPose(-35.3633280, 149.1652241, 0, 90)   # east, north, up, heading
enu_origin = ENUPose(0, 0, gra_origin.alt, gra_origin.heading) 

base_home = ENUPose(5,5,0,45)
base_path = Plan.create_square_path(side_len=10, alt=5,heading=0)

color =Color.BLUE

## Gazebo Congiguration

In [ ]:
gaz_config = ConfigGazebo(origin= enu_origin,
                          world_path="simulator/gazebo/worlds/runway.world")

gaz_config.add(base_path=base_path,base_home=base_home,color=color)
gaz_config.show()

## QGroundControl Configuration

In [ ]:
qgc_config = ConfigQGC(origin= gra_origin)

qgc_config.add(base_path=base_path,base_home=base_home,color=color)
qgc_config.show()

## No Visualizer

In [ ]:
novis_config = ConfigVis[int]()
novis_config.add_vehicle(1)

In [ ]:
novis=NoneVisualizer(novis_config)

gaz= Gazebo(gaz_config,gra_origin)

qgc=QGC(qgc_config)

## Launch Simulatotion

In [ ]:
# TODO: Is it possible/feasible use more than one visualizer at the same time? 
# It seems there is a port conflict, only the visualizer that launches first works.

simulator = Simulator(
	visualizers=[gaz],
	gcs_sysids={'GCS': [1]},
	missions=[veh.mission for veh in qgc_config.vehicles],
	verbose=2,
)
orac = simulator.launch()

## Oracle checking

In [ ]:
while len(orac.conns):
    for sysid in list(orac.conns.keys()):
        if orac.is_plan_done(sysid):
            orac.remove(sysid)